# Processing and analysis of data for meteor detection
## TABLES - Setting up the working environment for data tables creating

This command is used to install the Python packages matplotlib and pandas, which are commonly used for data visualization and data manipulation tasks, respectively.

NOTE: The package "opencv-python" is already included in the previous installation step.

In [ ]:
pip install opencv-python

These import statements bring in various Python libraries/modules needed for the subsequent code execution. Each import statement brings in specific functionality from the corresponding library/module.

In [ ]:
import torch
import os
import cv2
import time
import datetime
import csv
import PIL
import tensorflow as tf
import shutil
import pandas as pd
import numpy as np
import json
import math 
import scipy.io
import scipy.misc
from IPython.display import Image, clear_output
from matplotlib.pyplot import imshow
from PIL import Image, ImageDraw
from skimage.io import imread
from skimage.transform import resize
from tensorflow.keras.models import load_model, Model
from sys import path
from os import listdir
from os.path import isfile, join

Next code defines variables related to paths for image and text files used in testing. 

imagePath: Represents the directory path to the testing images along with their corresponding text files.

route: Refers to a specific route or directory within the Ultralytics structure.

rootPath: Constructs the root path for the labels based on the provided route.

path: Indicates the general path to the Ultralytics directory.

In [ ]:
imagePath="data/lightning/Kamila/testovacia"       
route="ALL_704_c10iou10_ult_train_final"
rootPath = r"ultralytics/runs/detect/predict/" + route + "/labels"
path = "ultralytics"

This code is responsible for parsing label files associated with images, extracting useful information from them, and storing that information in a structured format in these steps:

1. Defines paths to directories containing image and label files.
2. Creates a CSV file named 'testing5.csv' to store extracted information.
3. Reads through each label file in the specified directory.
4. Processes the content of each label file to extract relevant information such as event name, label, coordinates, width, height, confidence, duration, and date.
5. Writes the extracted information into the CSV file.

In [ ]:
img_cesta = os.getcwd() + "/ultralytics/runs/detect/predict/ALL_704_c10iou10_ult_train_final"
tab_cesta = os.getcwd() + "/ultralytics/runs/detect/predict/ALL_704_c10iou10_ult_train_final/labels"
name = [f for f in listdir(tab_cesta) if isfile(join(tab_cesta, f))]
csvOut = 'testing5.csv'
path = '/ultralytics/runs/detect/predict/ALL_704_c10iou10_ult_train_final/labels'

import csv
import os
from os import listdir
from os.path import isfile, join
from PIL import Image

with open(csvOut, mode='w') as csv_file:
    fieldnames = ['event name', 'label', 'x', 'y', 'width', 'height', 'confidence', 'length_in_sec', 'picture_ends_in', 'date']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for root, dirs, files in os.walk(tab_cesta):
        for name in files:
            file = os.getcwd() + path + "/" + name
            print(file)
            with open(file) as f:
                meno = os.path.splitext(name)[0]
                lines = f.readlines()
                dlzka = len(lines)
                i = 0
                while i < dlzka:
                    try:
                        image1 = Image.open(os.path.join(img_cesta, name.split(".")[0] + ".jpg"))
                        image2 = (os.path.join(img_cesta, name.split(".")[0])).split("/")[11].split("_")[1]
                        image3 = (os.path.join(img_cesta, name.split(".")[0])).split("/")[11].split("_")[0]
                    except:
                        break
                    line = lines[i]
                    splitted = line.split(" ")
                    list(map(lambda x: x.strip(), splitted))
                    length = round((float(splitted[3]) * (image1.size[0]) * (5 / 168.5)), 4)
                    begins = image2[0:2] + ":" + image2[2:4] + ":" + image2[4:6]
                    date = image3[11:] + "." + image3[9:11] + "." + image3[5:9]
                    if int(splitted[0]) == 0:
                        a = "krátka bez headecha"
                    if int(splitted[0]) == 1:
                        a = "krátka s headechom"
                    if int(splitted[0]) == 2:
                        a = "dlhá bez headecha"
                    if int(splitted[0]) == 3:
                        a = "dlhá s headechom"
                    writer.writerow({
                        'event name': meno,
                        'label': a,
                        'x': splitted[1],
                        'y': splitted[2],
                        'width': splitted[3],
                        'height': splitted[4],
                        'confidence': splitted[5],
                        'length_in_sec': length,
                        'picture_ends_in': begins,
                        'date': date})
                    i = i + 1

This code ensures that only unique detections are retained in the dataset by removing any duplicate entries based on specified conditions in these steps:

1. It reads the contents of a CSV file named 'testing5.csv'.
2. It compares each row with every other row in the CSV file to identify duplicates.
3. If two rows are considered duplicates based on specified criteria (e.g., similar coordinates and event name), it keeps the row with the higher confidence score.
4. It writes the non-duplicate rows to a new CSV file named 'testing5_1.csv', effectively removing duplicates from the dataset.

In [ ]:
import csv

csvfile = 'testing5.csv'
lines2=list()
zhoda=False

with open(csvfile, newline='') as f:
    reader = csv.reader(f)
    header = []
    header = next(reader)
    lines2.append(header)
    for row in reader:
        zhoda=False
        for compare_row in reader:
            if(row!=compare_row):
                if((float(row[2])-0.15 <= float(compare_row[2])) and (float(row[2])+0.15 >= float(compare_row[2])) and (float(row[3])-0.1 <= float(compare_row[3])) and (float(row[3])+0.1 >= float(compare_row[3])) and (row[0]==compare_row[0])):     
                    if((row[6])>(compare_row[6])):
                        lines2.append(row)
                    elif((row[6])<(compare_row[6])):
                        lines2.append(compare_row)
                    zhoda=True
            break
        if (zhoda==False):
            lines2.append(row)
            lines2.append(compare_row)

        
            


with open('testing5_1.csv', 'w', newline='') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(lines2)